In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Carregar o dataset
url = "https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv"
df = pd.read_csv(url)

#Análise inicial
print(df.head())
print(df.info())

# Pre-processamento
label_encoder = LabelEncoder()
df['sex'] = label_encoder.fit_transform(df['sex'])
df['smoker'] = label_encoder.fit_transform(df['smoker'])
df['region'] = label_encoder.fit_transform(df['region'])

# Separar features e target
X = df.drop('charges', axis=1)
y = df['charges']

# Dividir o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Modelo 1: Regressão Linear
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Modelo 2: Random Forest
param = {'n_estimators': [50, 100, 200], 'max_depth': [5, 10, 15]}
rf = RandomForestRegressor(random_state=42)
gridsearch = GridSearchCV(rf, param, cv=5, scoring='neg_mean_absolute_error')
gridsearch.fit(X_train, y_train)

# Melhor modelo e parametros
# Melhor modelo = Random Forest
best_rf = gridsearch.best_estimator_
print("Melhores parametros para o random forest:", gridsearch.best_params_)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Avaliação do modelo de regressão linear
y_pred_rl = lin_reg.predict(X_test)
mae_rl = mean_absolute_error(y_test, y_pred_rl)
rmse_rl = np.sqrt(mean_squared_error(y_test, y_pred_rl))
r2_rl = r2_score(y_test, y_pred_rl)

# Avaliação do modelo random forest
y_pred_rf = best_rf.predict(X_test)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

print("Regressão Linear:")
print("MAE:", mae_rl)
print("RMSE:", rmse_rl)
print("R²:", r2_rl)

print("\nRandom Forest:")
print("MAE:", mae_rf)
print("RMSE:", rmse_rf)
print("R²:", r2_rf)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Grafico de dispersão para o random forest
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred_rf)
plt.xlabel("Valores Reais")
plt.ylabel("Previsões")
plt.title("Gráfico de Dispersão - Modelo Random Forest")
plt.show()

In [ ]:
import joblib

# Salvar o modelo
joblib.dump(best_rf, 'random_forest_model.joblib')
joblib.dump(scaler, 'scaler.joblib')

In [ ]:
# Carregar o modelo para fazer previsões
loaded_model = joblib.load('random_forest_model.joblib')
loaded_scaler = joblib.load('scaler.joblib')

# Fazer previsão
sample_data = np.array([[25, 1, 30, 0, 1, 2]])  # Exemplo de entrada
sample_data_scaled = loaded_scaler.transform(sample_data)
prediction = loaded_model.predict(sample_data_scaled)
print("Previsão de custo de seguro:", prediction[0])